# Core 10 — Shutdown Event Contract (core10_01)

본 노트북은 “Design Shutdown”을 운영 시스템의 **공식 사건(event)** 으로 정의한다.

- Core 8/9에서 흩어진 shutdown 트리거를 Core 10에서 **단일 계약(contract)** 으로 봉인한다.
- shutdown은 실패가 아니라 정책 실행 결과이며, shutdown 이후에도 운영은 계속된다.

정책 문장:
- “Design ends, operation continues.”

산출물:
- core10_01_shutdown_contract.json

In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [2]:
# Inputs (존재하지 않는 파일은 optional로 둠)
CORE8_FALLBACK_DECISIONS = Path("../artifact/core8/core8_06_fallback_decisions.csv")
CORE9_RESERVATION_LOG    = Path("../artifact/core9/core9_04_reservation_log.csv")
CORE8_TRACE              = Path("../artifact/core8/core8_03_refusal_state_trace_counterfactual.csv")

assert CORE8_TRACE.exists(), "core8_03_refusal_state_trace_counterfactual.csv not found"

EXPORT_DIR = Path("../artifact/core10")
EXPORT_DIR.mkdir(exist_ok=True)

OUT_CONTRACT = EXPORT_DIR / "core10_01_shutdown_contract.json"
OUT_EVENTS   = EXPORT_DIR / "core10_01_shutdown_events.csv"

trace = pd.read_csv(CORE8_TRACE)

fallback = pd.read_csv(CORE8_FALLBACK_DECISIONS) if CORE8_FALLBACK_DECISIONS.exists() else None
reslog   = pd.read_csv(CORE9_RESERVATION_LOG) if CORE9_RESERVATION_LOG.exists() else None

print("loaded:", {
    "trace": len(trace),
    "fallback": None if fallback is None else len(fallback),
    "reservation": None if reslog is None else len(reslog)
}) #  Load Inputs

loaded: {'trace': 180, 'fallback': 180, 'reservation': 180}


In [3]:
KEY = ["run_id","case_id","antibody_id","step"]

for c in KEY:
    assert c in trace.columns, f"trace missing {c}"

trace["step"] = pd.to_numeric(trace["step"], errors="coerce").astype("Int64")

if fallback is not None:
    fallback = fallback.loc[:, ~fallback.columns.duplicated()].copy()
    for c in KEY:
        assert c in fallback.columns, f"fallback missing {c}"
    fallback["step"] = pd.to_numeric(fallback["step"], errors="coerce").astype("Int64")

if reslog is not None:
    reslog = reslog.loc[:, ~reslog.columns.duplicated()].copy()
    for c in KEY:
        assert c in reslog.columns, f"reservation log missing {c}"
    reslog["step"] = pd.to_numeric(reslog["step"], errors="coerce").astype("Int64") # Normalize minimal keys

## Shutdown Trigger Priority (Deterministic)

Shutdown은 “가장 강한 사건”이 먼저 발생한 시점으로 정의한다.

우선순위(높을수록 강함):
1) CORE8_REFUSAL            (fallback_stage == REFUSAL)
2) CORE8_FALLBACK_ENTER     (fallback_stage == FALLBACK_ENTER)
3) CORE9_RESERVATION_CONFIRMED (reservation_status == CONFIRMED)
4) CORE8_PARTIAL_SEAL       (fallback_stage == PARTIAL_SEAL)  *선택적*
5) NONE                     (어떤 shutdown 사건도 없음)

원칙:
- shutdown은 설계 시스템 종료 사건이며, 이후 운영 시스템은 계속 작동한다.
- shutdown_step은 위 우선순위에서 최초로 만족되는 step으로 고정된다.

In [4]:
CONTRACT = {
    "contract_id": "core10_01_shutdown_contract_v1",
    "policy_statement": "Design ends, operation continues.",
    "keys": KEY,
    "trigger_priority": [
        {"trigger": "CORE8_REFUSAL", "source": "core8_06_fallback_decisions.csv", "condition": "fallback_stage == 'REFUSAL'"},
        {"trigger": "CORE8_FALLBACK_ENTER", "source": "core8_06_fallback_decisions.csv", "condition": "fallback_stage == 'FALLBACK_ENTER'"},
        {"trigger": "CORE9_RESERVATION_CONFIRMED", "source": "core9_04_reservation_log.csv", "condition": "reservation_status == 'CONFIRMED'"},
        {"trigger": "CORE8_PARTIAL_SEAL", "source": "core8_06_fallback_decisions.csv", "condition": "fallback_stage == 'PARTIAL_SEAL'"},
        {"trigger": "NONE", "source": "derived", "condition": "no trigger matched"},
    ],
    "shutdown_step_rule": "For each (run_id, case_id, antibody_id), pick earliest step among triggers with highest priority.",
    "notes": [
        "This contract defines shutdown as an event, not a failure.",
        "Post-shutdown operation is handled in Core10 allocation policy."
    ]
}

CONTRACT # Define contract parameters

{'contract_id': 'core10_01_shutdown_contract_v1',
 'policy_statement': 'Design ends, operation continues.',
 'keys': ['run_id', 'case_id', 'antibody_id', 'step'],
 'trigger_priority': [{'trigger': 'CORE8_REFUSAL',
   'source': 'core8_06_fallback_decisions.csv',
   'condition': "fallback_stage == 'REFUSAL'"},
  {'trigger': 'CORE8_FALLBACK_ENTER',
   'source': 'core8_06_fallback_decisions.csv',
   'condition': "fallback_stage == 'FALLBACK_ENTER'"},
  {'trigger': 'CORE9_RESERVATION_CONFIRMED',
   'source': 'core9_04_reservation_log.csv',
   'condition': "reservation_status == 'CONFIRMED'"},
  {'trigger': 'CORE8_PARTIAL_SEAL',
   'source': 'core8_06_fallback_decisions.csv',
   'condition': "fallback_stage == 'PARTIAL_SEAL'"},
  {'trigger': 'NONE', 'source': 'derived', 'condition': 'no trigger matched'}],
 'shutdown_step_rule': 'For each (run_id, case_id, antibody_id), pick earliest step among triggers with highest priority.',
 'notes': ['This contract defines shutdown as an event, not a fa

In [5]:
# start from trace skeleton to ensure all rows exist
base = trace[KEY].dropna().copy()

# attach fallback stage if exists
if fallback is not None:
    keep_fb = [c for c in KEY + ["fallback_stage","fallback_reason_code","fallback_score"] if c in fallback.columns]
    base = base.merge(fallback[keep_fb], on=KEY, how="left")

# attach reservation status if exists
if reslog is not None:
    keep_rs = [c for c in KEY + ["reservation_status","reservation_reason_code","risk_score_total"] if c in reslog.columns]
    base = base.merge(reslog[keep_rs], on=KEY, how="left")

base.head() # Build unified event table

,run_id,case_id,antibody_id,step,fallback_stage,fallback_reason_code,fallback_score,reservation_status,reservation_reason_code,risk_score_total
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.000,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,HOLD,REASON_MIN_STEPS_NOT_REACHED,0.006,HOLD,REASON_MIN_STEPS_NOT_REACHED,NaN


In [6]:
base["is_core8_refusal"] = base.get("fallback_stage", pd.Series([None]*len(base))).eq("REFUSAL") if "fallback_stage" in base.columns else False
base["is_core8_fallback_enter"] = base.get("fallback_stage", pd.Series([None]*len(base))).eq("FALLBACK_ENTER") if "fallback_stage" in base.columns else False
base["is_core9_res_confirmed"] = base.get("reservation_status", pd.Series([None]*len(base))).eq("CONFIRMED") if "reservation_status" in base.columns else False
base["is_core8_partial_seal"] = base.get("fallback_stage", pd.Series([None]*len(base))).eq("PARTIAL_SEAL") if "fallback_stage" in base.columns else False

# priority score: smaller = stronger (for deterministic min)
# refusal(0) > fallback_enter(1) > reservation_confirmed(2) > partial_seal(3) > none(999)
def priority_row(r):
    if r["is_core8_refusal"]:
        return 0
    if r["is_core8_fallback_enter"]:
        return 1
    if r["is_core9_res_confirmed"]:
        return 2
    if r["is_core8_partial_seal"]:
        return 3
    return 999

base["shutdown_priority"] = base.apply(priority_row, axis=1)

def trigger_name_from_priority(p):
    return {
        0: "CORE8_REFUSAL",
        1: "CORE8_FALLBACK_ENTER",
        2: "CORE9_RESERVATION_CONFIRMED",
        3: "CORE8_PARTIAL_SEAL",
        999: "NONE"
    }.get(int(p), "NONE")

base["shutdown_trigger"] = base["shutdown_priority"].map(trigger_name_from_priority) # Compute trigger flags

In [7]:
def compute_shutdown_for_group(g: pd.DataFrame) -> pd.DataFrame:
    g = g.sort_values("step").copy()

    # 후보: priority < 999 인 row들 중
    candidates = g[g["shutdown_priority"] < 999]
    if len(candidates) == 0:
        g["shutdown_step"] = np.nan
        g["shutdown_trigger_final"] = "NONE"
        return g

    # strongest priority 먼저
    best_p = int(candidates["shutdown_priority"].min())
    best_rows = candidates[candidates["shutdown_priority"] == best_p]

    # 그 중 earliest step
    shutdown_step = int(best_rows["step"].min())
    g["shutdown_step"] = shutdown_step
    g["shutdown_trigger_final"] = trigger_name_from_priority(best_p)
    return g

shutdown_df = (
    base.groupby(["run_id","case_id","antibody_id"], group_keys=False)
        .apply(compute_shutdown_for_group)
        .reset_index(drop=True)
)

shutdown_df[["run_id","case_id","antibody_id","step","shutdown_trigger","shutdown_trigger_final","shutdown_step"]].head(30) # Compute shutdown_step per trajectory

/var/folders/sc/_dk299z14xs6j6n9ggl_ny5r0000gn/T/ipykernel_25575/3556529387.py:23: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_shutdown_for_group)


,run_id,case_id,antibody_id,step,shutdown_trigger,shutdown_trigger_final,shutdown_step
0,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,0,NONE,NONE,NaN
1,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,1,NONE,NONE,NaN
2,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,2,NONE,NONE,NaN
3,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,3,NONE,NONE,NaN
4,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,4,NONE,NONE,NaN
5,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,5,NONE,NONE,NaN
6,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,6,NONE,NONE,NaN
7,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,7,NONE,NONE,NaN
8,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,8,NONE,NONE,NaN
9,core7_04_1767776352,A_ALWAYS_ALLOW,antibody_A,9,NONE,NONE,NaN


In [8]:
# “사건” row만 남김: shutdown_step에 해당하는 row
events = shutdown_df.copy()

events = events[
    (events["shutdown_trigger_final"] != "NONE") &
    (events["step"] == events["shutdown_step"])
].copy()

events_cols = [c for c in [
    "run_id","case_id","antibody_id",
    "shutdown_step","shutdown_trigger_final",
    "fallback_stage","fallback_reason_code","fallback_score",
    "reservation_status","reservation_reason_code","risk_score_total"
] if c in events.columns]

events = events[events_cols].sort_values(["run_id","case_id","antibody_id"])
events.head(50), events.shape # Create compact shutdown event table

OUT_CONTRACT.write_text(json.dumps(CONTRACT, indent=2), encoding="utf-8")
events.to_csv(OUT_EVENTS, index=False)

print("Exported:")
print("-", OUT_CONTRACT)
print("-", OUT_EVENTS)

Exported:
- ../artifact/core10/core10_01_shutdown_contract.json
- ../artifact/core10/core10_01_shutdown_events.csv


## Sanity Check

- shutdown 사건이 있는 trajectory 비율
- trigger 우선순위 분포
- A_ALWAYS_ALLOW vs B_GOVERNED에서 shutdown trigger가 어떻게 다른지

In [9]:
if "case_id" in events.columns:
    events["bucket"] = events["case_id"].astype(str).map(lambda x: "A_ALWAYS_ALLOW" if x.startswith("A_") else ("B_GOVERNED" if x.startswith("B_") else "OTHER"))

summary = (
    events.groupby(["bucket","shutdown_trigger_final"], as_index=False)
          .size()
          .sort_values(["bucket","size"], ascending=[True, False])
)

summary

,bucket,shutdown_trigger_final,size
0,B_GOVERNED,CORE8_PARTIAL_SEAL,1
